## Note to Jack
I'm a physicist, not a CS-trained person, be easy on me today =)

In [ ]:
import pytest
import ipytest

ipytest.autoconfig()

## Functions

In [ ]:
def parse_session(session):

    tree = {'/':{
        'files': {}
        }
    }
    pwd = ['']
    dir_list = False

    for line in session:
        line = line.split()
        
        # Commands
        if line[0] == '$':
            dir_list = False 
            if line[1] == 'cd':

                # temp
                new_dir = pwd.copy()
                new_dir.append(line[2])

                # Special root case
                if line[2] == '/':
                    pwd = ['']
                # Open parent
                elif line[2] == '..':
                    pwd.pop()
                # Directory already added to tree
                elif '/'.join(new_dir) in tree:
                    # Open dir
                    pwd = new_dir
                # Add directory to tree
                else:
                    # Create dir
                    tree['/'.join(new_dir)] = {
                        'files': {}
                    }
                    # Now open it
                    pwd = new_dir
        # Useless command (nothing to do here)
        elif line[0] == 'ls': 
            pass
        elif line[0] == 'dir':
            # temp
            new_dir = pwd.copy()
            new_dir.append(line[1])
            
            # Create dir but don't open it
            if '/'.join(new_dir) not in tree:
                tree['/'.join(new_dir)] = {
                    'files': {}
                }
        # It's a file! Fill in the info
        else:
            # temp
            if pwd == ['']:
                path = '/'
            else:
                tmp_dir = pwd.copy()
                path = '/'.join(tmp_dir)
            tree[path]['files'][line[1]] = line[0]
            
    return tree

def parse_tree_size(tree):
    size = 0
    cutoff = 100000
    for dir in tree:
        dir_size = 0
        for file in tree[dir]['files']:
            dir_size += int(tree[dir]['files'][file])

        # Repeat for subdirectories
        for subdir in tree:
            if subdir == dir:
                pass
            # Is an actual subdirectory
            elif subdir.startswith(dir):
                for file in tree[subdir]['files']:
                    dir_size += int(tree[subdir]['files'][file])

        if dir_size <= cutoff:
            size+=dir_size
        
    return size

def find_dir_min_usage(tree):
    for dir in tree:
        dir_size = 0
        for file in tree[dir]['files']:
            dir_size += int(tree[dir]['files'][file])

        # Repeat for subdirectories
        for subdir in tree:
            if subdir == dir:
                pass
            # Is an actual subdirectory
            elif subdir.startswith(dir):
                for file in tree[subdir]['files']:
                    dir_size += int(tree[subdir]['files'][file])

        tree[dir]['du'] = dir_size

    system_size = 70000000
    system_usage = system_size-tree['/']['du']  
    free_space_needed = 30000000
    min_cutoff = free_space_needed - system_usage
    dir_name = ''
    size = 0
    for dir in tree:
        if tree[dir]['du']>min_cutoff:
            if size == 0 or tree[dir]['du']<size:
                dir_name = dir
                size = tree[dir]['du']  
    return size, dir_name

## Run tests

In [ ]:
# define the tests
class TestClass:
    def setup_method(self):
        self.session = open('test_input.txt').read().splitlines()

    def test_read_inital_line(self):
        assert self.session[0] == '$ cd /'

    def test_expected_dirs(self):
        tree = parse_session(self.session)

        for dir in tree:
            assert dir in ['/', '/a', '/a/e', '/d']
    def test_expected_files_for_dir(self):
        tree = parse_session(self.session)
        dir = '/a/e'
        
        file = 'i'
        assert tree[dir]['files'][file] == '584'
    
    def test_expected_parse_tree_size(self):
        tree = parse_session(self.session)
        size = parse_tree_size(tree)
        assert size == 95437

    def test_find_dir_min_usage(self):
        tree = parse_session(self.session)
        size, dir_name = find_dir_min_usage(tree)
        assert size == 24933642
        assert dir_name == '/d'

ipytest.run()

## Part 1

In [ ]:

## Read input
session = open('input.txt').read().splitlines()

In [ ]:
# Parse session into a tree
tree  = parse_session(session)

In [ ]:
# Parse the tree dir size
parse_tree_size(tree)

## Part 2

In [ ]:
size, dir_name = find_dir_min_usage(tree)
print(size, dir_name)